In [ ]:
from pathlib import Path

import os


from dotenv import load_dotenv

from azure.identity import AzureAuthorityHosts

from azure.search.documents import SearchClient


env_directory = Path.cwd() / ".azure"

env_file = next(
    (candidate / ".env" for candidate in env_directory.glob("avcoe-*")),
    None,
)

if env_file is None:
    raise FileNotFoundError(
        "Could not locate an avcoe-* environment directory under .azure"
    )


load_dotenv(dotenv_path=env_file, override=False)


search_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]

search_index = os.environ["SEARCH_INDEX_NAME"]

print(f"Using search endpoint {search_endpoint}, and index: {search_index}")

authority_host = (
    AzureAuthorityHosts.AZURE_GOVERNMENT
    if os.getenv("CLOUD_NAME") == "AzureUSGovernment"
    else AzureAuthorityHosts.AZURE_PUBLIC_CLOUD
)

print(f"Using authority host: {authority_host}")

In [ ]:
# For Azure Government, we need to use AzureCliCredential directly with the right scope
from azure.identity import AzureCliCredential

# Set the correct credential scope for Azure Government
credential_scope = (
    "https://search.azure.us/.default"
    if os.getenv("CLOUD_NAME") == "AzureUSGovernment"
    else "https://search.azure.com/.default"
)

print(f"Using credential scope: {credential_scope}")

# Use AzureCliCredential directly since we know CLI is authenticated properly
credential = AzureCliCredential()

# Create a wrapper that uses the right scope


class ScopedCredential:
    def __init__(self, credential, scope):
        self._credential = credential
        self._scope = scope

    def get_token(self, *scopes, **kwargs):
        # Always use our custom scope regardless of what's requested
        return self._credential.get_token(self._scope, **kwargs)


scoped_credential = ScopedCredential(credential, credential_scope)

# Create SearchClient with the scoped credential
search_client = SearchClient(search_endpoint, search_index, scoped_credential)

In [ ]:
results = search_client.search("test")

In [ ]:
for result in results:
    print(result)
    break